# Kaggle Learn-and-compete `Poker Rule Induction`

## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from functools import reduce
from itertools import permutations

#### Read data

In [31]:
test_dataset = pd.read_csv('test.csv', delimiter=',')

In [32]:
test_dataset

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,1,1,10,2,2,3,3,3,8,1,1
1,2,2,13,3,5,3,7,4,6,1,4
2,3,1,3,1,11,2,8,2,1,2,4
3,4,1,6,3,3,4,7,1,8,3,11
4,5,2,10,3,4,1,6,2,12,2,6
5,6,1,4,3,10,2,11,2,6,1,7
6,7,1,10,3,8,1,4,3,11,3,9
7,8,2,11,3,8,1,1,1,11,2,3
8,9,3,4,1,1,1,3,3,5,3,6
9,10,3,12,2,1,1,3,1,2,3,10


In [2]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [3]:
train_dataset['hand'].value_counts()

0    12493
1    10599
2     1206
3      513
4       93
5       54
6       36
7        6
9        5
8        5
Name: hand, dtype: int64

##### Data augmentation

In [4]:
def repeat(data, times):
    repeated = data.copy()
    for i in range(times - 1):
        repeated = pd.concat([repeated, data], axis=0)
    return repeated

def data_augmentation(data):
    data_aug = data.copy()
    counts = data['hand'].value_counts()
    for value, count in enumerate(counts):
        if value == 0:
            continue
        else:
            times = counts[0]//count
        repeated = repeat(data[data['hand'] == value], times)
        data_aug = pd.concat([data_aug, repeated], axis=0)
    data_aug.index = list(range(data_aug.shape[0]))
    return data_aug

In [5]:
train_dataset_aug = data_augmentation(train_dataset)

In [8]:
train_dataset_aug['hand'].value_counts()

1    21198
2    13266
3    12825
4    12555
6    12528
5    12528
7    12498
9    12495
8    12495
0    12493
Name: hand, dtype: int64

In [34]:
data = train_dataset[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']]
labels = train_dataset[['hand']]

In [35]:
data[0:10]

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5
0,4,9,2,1,2,2,4,7,2,8
1,1,4,3,6,1,12,3,11,2,7
2,1,11,4,1,3,7,4,11,2,1
3,2,9,2,4,3,6,1,9,4,9
4,1,8,2,4,2,11,2,2,2,1
5,2,5,1,5,2,13,2,3,3,13
6,3,10,4,6,1,4,2,13,4,5
7,4,10,3,1,2,13,4,2,4,7
8,3,2,4,10,3,3,4,4,1,9
9,2,7,3,8,4,8,2,13,2,12


In [36]:
labels_vect = tf.keras.utils.to_categorical(labels)

#### One hot encoding

In [8]:
def data_hotencode(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

def hand_hotencode(hand, columns, encoder):
    df = pd.DataFrame(data=[hand], columns=['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5'])
    return data_hotencode(df, columns, encoder, 'transform')

In [10]:
encoder = OneHotEncoder()
columns_to_hotencode = ['S1', 'S2', 'S3', 'S4', 'S5']
data_encoded = data_hotencode(data, columns_to_hotencode, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 25)


,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Sort

In [61]:
def sort(data, columns, ace_is_max = False):
    sorted = data[columns]
    if ace_is_max:
        sorted = sorted.replace(1, 14) - 1
    else:
        sorted = sorted
    
    sorted.values.sort()
    sorted['C6'] = 14 * (sorted['C1'] == 1)
    sorted.values.sort()
    data_sorted = pd.concat([sorted, data.drop(columns, axis=1)], axis=1)
    return data_sorted

In [62]:
columns_to_sort = ['C1', 'C2', 'C3', 'C4', 'C5']
data_sorted = sort(data_encoded, columns_to_sort)
data_sorted.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,C1,C2,C3,C4,C5,C6,0,1,2,3,...,10,11,12,13,14,15,16,17,18,19
0,1,2,7,8,9,14,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0,4,6,7,11,12,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,1,7,11,11,14,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0,4,6,9,9,9,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,2,4,8,11,14,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [63]:
def normalize(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data.drop(columns, axis=1), data_normalized], axis=1)
    to_return.columns = list(range(to_return.shape[1]))
    return to_return

In [64]:
columns_to_normalize = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
mean = data_sorted[columns_to_sort].mean().mean()
std = pd.Series(data_sorted[columns_to_sort].values.flatten()).std()
data_normalized = normalize(data_sorted, columns_to_normalize, mean, std)
data_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-1.150949,-0.893612,0.393078,0.650416,0.907753,2.194443
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,-1.408287,-0.378936,0.135740,0.393078,1.422429,1.679767
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-1.150949,-1.150949,0.393078,1.422429,1.422429,2.194443
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-1.408287,-0.378936,0.135740,0.907753,0.907753,0.907753
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-1.150949,-0.893612,-0.378936,0.650416,1.422429,2.194443


#### Test function

In [65]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [66]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


### Solutions

1. Neural network
2. Random forest

Ousiders:
 - Genetic algorithm
 - Decision tree

### Neural Network

In [67]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(70, activation='relu', input_shape=(26,)))
model.add(keras.layers.Dropout(0.1))
#hidden
model.add(keras.layers.Dense(40, activation='relu'))
model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(20, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=256)

Epoch 1/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 19us/step - loss: 1.4467 - acc: 0.4504

Epoch 2/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.9993 - acc: 0.5078

Epoch 3/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.9742 - acc: 0.5274

Epoch 4/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.9587 - acc: 0.5471

Epoch 5/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.9467 - acc: 0.5587

Epoch 6/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 0.9361 - acc: 0.5666

Epoch 7/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0243 - acc: 0.9943

Epoch 112/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 16us/step - loss: 0.0226 - acc: 0.9946

Epoch 113/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 16us/step - loss: 0.0237 - acc: 0.9944

Epoch 114/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 14us/step - loss: 0.0199 - acc: 0.9944

Epoch 115/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 0.0206 - acc: 0.9946

Epoch 116/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.0233 - acc: 0.9945

Epoch 117/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.0045 - acc: 0.9990

Epoch 222/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0051 - acc: 0.9988

Epoch 223/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0049 - acc: 0.9990

Epoch 224/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0045 - acc: 0.9990

Epoch 225/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.0052 - acc: 0.9983

Epoch 226/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0050 - acc: 0.9990

Epoch 227/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 0s 16us/step - loss: 0.0031 - acc: 0.9994

Epoch 332/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 15us/step - loss: 0.0018 - acc: 0.9996

Epoch 333/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 15us/step - loss: 0.0032 - acc: 0.9992

Epoch 334/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 14us/step - loss: 0.0024 - acc: 0.9994

Epoch 335/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 15us/step - loss: 0.0020 - acc: 0.9994

Epoch 336/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 15us/step - loss: 0.0024 - acc: 0.9992

Epoch 337/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 15us/step - loss: 

25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0013 - acc: 0.9997

Epoch 442/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 15us/step - loss: 0.0012 - acc: 0.9996

Epoch 443/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0024 - acc: 0.9993

Epoch 444/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0014 - acc: 0.9997

Epoch 445/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 0.0027 - acc: 0.9995

Epoch 446/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.0013 - acc: 0.9996

Epoch 447/500
25010/25010 [==============================]25010/25010 [==============================] - 0s 13us/step - loss: 

In [68]:
y_test = test_dataset.drop(['id'], axis=1).apply(calc_hand_label, axis=1)

In [39]:
model.save('./model-sort-166')

### Predictions

In [69]:
test_dataset_noid = test_dataset.drop(['id'], axis=1)

In [70]:
test_sample = test_dataset_noid
test_data_sample_encoded = normalize(
    sort(
        data_hotencode(test_sample, columns_to_hotencode, encoder),
        columns_to_sort,
        False,
    ), 
    columns_to_normalize,
    mean,
    std
)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [1]:
model1 = keras.models.load_model('./model-sort-166')
model1.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

NameError: name 'keras' is not defined

In [71]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_sample_encoded))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [72]:
#test_data_sample_predicted = np.maximum(test_data_sample_predicted_1, test_data_sample_predicted_2)
#test_data_sample_predicted = test_data_sample_predicted_2

test_data_sample_labels = y_test
df = pd.DataFrame({
    'predicted': test_data_sample_predicted, 
    'real': test_data_sample_labels
})
df = pd.concat([test_sample, df], axis=1)
wrong_answers = df[df['predicted'] != df['real']]
print(wrong_answers.shape)
wrong_answers

(195, 12)


,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predicted,real
665,3,4,3,5,3,3,3,1,3,2,5,8
3902,4,4,2,3,3,2,3,1,1,5,1,4
9256,2,5,1,2,3,3,1,1,1,4,0,4
13976,4,7,3,7,2,7,4,1,2,1,2,6
14357,3,5,4,1,4,3,4,4,3,2,0,4
14657,4,1,1,5,3,4,3,3,1,2,1,4
15467,2,1,3,1,1,1,4,1,4,12,3,7
31539,3,1,4,1,1,1,3,4,2,1,3,7
31631,1,11,3,1,2,11,1,1,3,11,2,6
42427,2,3,3,1,3,2,1,5,3,4,0,4


In [73]:
wrong_answers['real'].value_counts()

4    63
7    49
0    40
6    36
8     7
Name: real, dtype: int64

In [48]:
wrong_answers.drop(['predicted', 'real'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

34        K♣, Q♣, 10♦, A♦, J♥
665        4♣, 5♣, 3♣, A♣, 2♣
3902       4♠, 3♦, 2♣, A♣, 5♥
9503       4♥, A♠, 2♠, 5♣, 3♣
12027     K♦, J♦, 10♦, Q♥, A♥
14357      5♣, A♠, 3♠, 4♠, 2♣
14657      A♠, 5♥, 4♣, 3♣, 2♥
15059     K♦, Q♥, A♥, 10♥, J♠
15877     10♦, Q♥, K♥, A♥, J♠
20151      4♣, A♣, 3♣, 5♦, 2♦
36911     K♦, J♣, A♣, 10♦, Q♥
42427      3♦, A♣, 2♣, 5♥, 4♣
48362     Q♦, J♦, A♠, 10♥, K♦
50832     Q♦, 10♥, A♦, K♥, J♠
51428      A♦, 4♣, 5♠, 3♥, 2♥
57633     J♠, Q♦, K♠, 10♠, A♠
65236     10♥, Q♣, A♦, K♦, J♠
66296     K♠, Q♣, J♥, 10♦, A♥
69065     Q♣, J♥, A♦, 10♦, K♦
70575     9♣, 8♥, K♦, J♠, 10♠
72500      3♠, 5♠, 2♣, A♣, 4♥
78123     10♦, A♥, J♥, K♣, Q♥
85179     Q♦, K♣, A♣, J♦, 10♥
88839     10♣, J♥, Q♦, A♥, K♥
91839      5♣, A♣, 4♠, 3♣, 2♦
95994     10♣, K♥, Q♥, J♠, A♦
96353     J♣, 9♥, 10♦, K♦, 8♠
97192      5♠, 3♣, 4♠, 2♦, A♣
100764     3♣, 5♣, 2♣, 4♥, A♣
103163    K♥, A♣, Q♦, J♣, 10♥
                 ...         
879365    K♣, A♥, 10♥, Q♦, J♦
890194    9♣, J♥, Q♣, 10♣, A♥
893648    